In [2]:
# # Helper functions for the parallel computing 
import pandas as pd
import numpy as np

import random
import os
import sys
import psutil

import matplotlib 
import matplotlib.pyplot as plt
import math
from multiprocessing import cpu_count,Pool 
import multiprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pygeohash as gh
import pickle
from datetime import date

import pysolar
from pysolar.solar import *
from datetime import date
import datetime as dt
import pygeohash as gh
from pvlib import atmosphere
import pvlib.solarposition as pv
from pvlib.tools import datetime_to_djd, djd_to_datetime
import pytz
import calendar
import numpy as np



class WithExtraArgs(object):
     def __init__(self, func, **args):
         self.func = func
         self.args = args
     def __call__(self, df):
         return self.func(df, **self.args)

def applyParallel(data, func,pool,partition,kwargs):
    data_split = [data[i:i + partition] for i in range(0, len(data), partition)]
    data =pool.map(WithExtraArgs(func, **kwargs), data_split)
    return data

In [4]:
city='new_method/hann/clustering/gridgrowing6'

data_acc=pd.read_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/train_acc.csv",header=0)

data_nonacc=pd.read_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/train_nonacc.csv",header=0)
train_minus1_nonacc=data_nonacc.loc[data_nonacc['cluster_id']==-2]
train_non_acc_array=train_minus1_nonacc.geohash.unique()

hann_test_nonacc=pd.read_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/test_nonacc.csv",header=0)
test_minus1_nonacc=hann_test_nonacc.loc[hann_test_nonacc['cluster_id']==-2]
test_nonacc_array=test_minus1_nonacc.geohash.unique()


hann_test_acc=pd.read_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/test_acc.csv",header=0)
test_minus1_acc=hann_test_acc.loc[hann_test_acc['cluster_id']==-2]
test_acc_array=test_minus1_acc.geohash.unique()
new_array = np.unique(np.concatenate((train_non_acc_array,test_nonacc_array,test_acc_array),0))


In [336]:
data_acc=pd.read_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/train_acc.csv",header=0)
data_acc

,acc_id,acc_lat,acc_long,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,STRZUSTAND,UART,UTYP1,geohash,sessiondivided,geohash7,Gridcluster_id
0,9867_2017,52.404062,9.646150,2017,1,11,4,0,6,4,u1qf5e,10-12,u1qf5eb,1224
1,15364_2017,52.403888,9.646154,2017,5,8,3,1,5,2,u1qf5e,8-10,u1qf5eb,1224
2,15437_2017,52.401917,9.646246,2017,5,17,2,0,5,3,u1qf5e,16-18,u1qf5e2,1224
3,16196_2017,52.399327,9.646483,2017,6,15,5,0,5,3,u1qf5e,14-16,u1qf5e0,1224
4,18217_2017,52.403573,9.646172,2017,8,18,3,0,5,3,u1qf5e,18-20,u1qf5eb,1224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5659,232958_2019,52.411138,9.696425,2019,5,14,6,0,5,3,u1qfht,14-16,u1qfhth,1402
5660,232983_2019,52.368419,9.718043,2019,5,19,5,0,0,7,u1qcvj,18-20,u1qcvjk,165
5661,232984_2019,52.382647,9.710951,2019,5,18,5,0,6,1,u1qcuz,18-20,u1qcuzz,165
5662,233021_2019,52.375711,9.750454,2019,5,7,4,0,2,6,u1qcvy,6-8,u1qcvys,165


In [337]:
df_neg=pd.DataFrame(new_array)
df_neg=df_neg.drop_duplicates()
df_neg

,0
0,u1qcg6
1,u1qcg7
2,u1qcgd
3,u1qcge
4,u1qcgg
...,...
274,u1r41k
275,u1r41m
276,u1r41n
277,u1r41p


In [338]:
data_acc.Gridcluster_id.max()

2341

In [340]:
cluster_id=data_acc.Gridcluster_id.max()+1
for index, row in df_neg.iterrows():
    df_neg.loc[index,'Gridcluster_id'] =cluster_id
    cluster_id=cluster_id+1

In [341]:
df_neg.columns=['geohash','cluster_id']
df_neg.to_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/minus1geohash.csv",index=False)
df_neg

,geohash,cluster_id
0,u1qcg6,2342.0
1,u1qcg7,2343.0
2,u1qcgd,2344.0
3,u1qcge,2345.0
4,u1qcgg,2346.0
...,...,...
274,u1r41k,2616.0
275,u1r41m,2617.0
276,u1r41n,2618.0
277,u1r41p,2619.0


In [344]:
#city1='new_method/hann/clustering/gridgrowing6'
data_acc=pd.read_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/train_acc.csv",header=0)
#data_acc.columns=['acc_id','geohash','cluster_id','acc_long','acc_lat']
data_acc

,acc_id,acc_lat,acc_long,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,STRZUSTAND,UART,UTYP1,geohash,sessiondivided,geohash7,Gridcluster_id
0,9867_2017,52.404062,9.646150,2017,1,11,4,0,6,4,u1qf5e,10-12,u1qf5eb,1224
1,15364_2017,52.403888,9.646154,2017,5,8,3,1,5,2,u1qf5e,8-10,u1qf5eb,1224
2,15437_2017,52.401917,9.646246,2017,5,17,2,0,5,3,u1qf5e,16-18,u1qf5e2,1224
3,16196_2017,52.399327,9.646483,2017,6,15,5,0,5,3,u1qf5e,14-16,u1qf5e0,1224
4,18217_2017,52.403573,9.646172,2017,8,18,3,0,5,3,u1qf5e,18-20,u1qf5eb,1224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5659,232958_2019,52.411138,9.696425,2019,5,14,6,0,5,3,u1qfht,14-16,u1qfhth,1402
5660,232983_2019,52.368419,9.718043,2019,5,19,5,0,0,7,u1qcvj,18-20,u1qcvjk,165
5661,232984_2019,52.382647,9.710951,2019,5,18,5,0,6,1,u1qcuz,18-20,u1qcuzz,165
5662,233021_2019,52.375711,9.750454,2019,5,7,4,0,2,6,u1qcvy,6-8,u1qcvys,165


In [346]:
data_nonacc

,geohash7,nonacc_lat,nonacc_long,UJAHR,UMONAT,UWOCHENTAG,hour,geohash,cluster_id
0,u1qcuyd,52.375183,9.704767,2017,9,3,18,u1qcuy,165.0
1,u1qcu96,52.345783,9.693942,2017,7,1,10,u1qcu9,130.0
2,u1r4095,52.388620,9.870285,2018,11,5,21,u1r409,-1.0
3,u1qfpx5,52.422443,9.827235,2017,1,3,23,u1qfpx,2143.0
4,u1qfr4y,52.441962,9.808172,2018,1,6,16,u1qfr4,-1.0
...,...,...,...,...,...,...,...,...,...
16987,u1qcwz4,52.333964,9.792531,2017,5,2,4,u1qcwz,807.0
16988,u1qcv51,52.356013,9.714025,2019,1,1,17,u1qcv5,165.0
16989,u1r40sh,52.405959,9.871585,2017,9,5,9,u1r40s,2302.0
16990,u1qcy8c,52.344068,9.779943,2018,1,5,23,u1qcy8,938.0


In [347]:
data=pd.read_csv('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/train_acc.csv',header=0)
data

,acc_id,acc_lat,acc_long,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,STRZUSTAND,UART,UTYP1,geohash,sessiondivided,geohash7,Gridcluster_id
0,9867_2017,52.404062,9.646150,2017,1,11,4,0,6,4,u1qf5e,10-12,u1qf5eb,1224
1,15364_2017,52.403888,9.646154,2017,5,8,3,1,5,2,u1qf5e,8-10,u1qf5eb,1224
2,15437_2017,52.401917,9.646246,2017,5,17,2,0,5,3,u1qf5e,16-18,u1qf5e2,1224
3,16196_2017,52.399327,9.646483,2017,6,15,5,0,5,3,u1qf5e,14-16,u1qf5e0,1224
4,18217_2017,52.403573,9.646172,2017,8,18,3,0,5,3,u1qf5e,18-20,u1qf5eb,1224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5659,232958_2019,52.411138,9.696425,2019,5,14,6,0,5,3,u1qfht,14-16,u1qfhth,1402
5660,232983_2019,52.368419,9.718043,2019,5,19,5,0,0,7,u1qcvj,18-20,u1qcvjk,165
5661,232984_2019,52.382647,9.710951,2019,5,18,5,0,6,1,u1qcuz,18-20,u1qcuzz,165
5662,233021_2019,52.375711,9.750454,2019,5,7,4,0,2,6,u1qcvy,6-8,u1qcvys,165


# for assiginging -1 cluster to their repsective geohash

In [348]:
def geohash(lat,long):
    geo=gh.encode(float(lat), float(long), precision=6)
    return geo
data_nonacc['geohash'] = data_nonacc.apply(lambda row: geohash(row['nonacc_lat'],row['nonacc_long']), axis=1)
data_nonacc

,geohash7,nonacc_lat,nonacc_long,UJAHR,UMONAT,UWOCHENTAG,hour,geohash,cluster_id
0,u1qcuyd,52.375183,9.704767,2017,9,3,18,u1qcuy,165.0
1,u1qcu96,52.345783,9.693942,2017,7,1,10,u1qcu9,130.0
2,u1r4095,52.388620,9.870285,2018,11,5,21,u1r409,-1.0
3,u1qfpx5,52.422443,9.827235,2017,1,3,23,u1qfpx,2143.0
4,u1qfr4y,52.441962,9.808172,2018,1,6,16,u1qfr4,-1.0
...,...,...,...,...,...,...,...,...,...
16987,u1qcwz4,52.333964,9.792531,2017,5,2,4,u1qcwz,807.0
16988,u1qcv51,52.356013,9.714025,2019,1,1,17,u1qcv5,165.0
16989,u1r40sh,52.405959,9.871585,2017,9,5,9,u1r40s,2302.0
16990,u1qcy8c,52.344068,9.779943,2018,1,5,23,u1qcy8,938.0


In [349]:
data_minus=data_nonacc.loc[data_nonacc['cluster_id']==-2]
data_minus

,geohash7,nonacc_lat,nonacc_long,UJAHR,UMONAT,UWOCHENTAG,hour,geohash,cluster_id
2,u1r4095,52.388620,9.870285,2018,11,5,21,u1r409,-1.0
4,u1qfr4y,52.441962,9.808172,2018,1,6,16,u1qfr4,-1.0
5,u1qfn37,52.389689,9.771591,2017,8,3,1,u1qfn3,-1.0
7,u1qf556,52.401003,9.627444,2019,1,5,4,u1qf55,-1.0
15,u1r1c5v,52.360680,9.895085,2018,4,6,13,u1r1c5,-1.0
...,...,...,...,...,...,...,...,...,...
16972,u1r40ng,52.420063,9.848290,2018,6,7,2,u1r40n,-1.0
16973,u1r18xs,52.337421,9.871995,2017,9,1,9,u1r18x,-1.0
16978,u1r4102,52.385001,9.887781,2017,3,4,13,u1r410,-1.0
16979,u1r40z0,52.421872,9.877405,2017,6,5,17,u1r40z,-1.0


In [ ]:
data_others=data_nonacc.loc[data_nonacc['cluster_id']>-2]
data_others

In [356]:
data_geohash=data_others[['geohash','cluster_id']]
data_geohash=data_geohash.drop_duplicates()
data_geohash=data_geohash.reset_index(drop=True)
data_geohash

,geohash,cluster_id
0,u1qcuy,165.0
1,u1qcu9,130.0
2,u1qfpx,2143.0
3,u1qcu1,87.0
4,u1qfp9,2071.0
...,...,...
1379,u1qcx7,813.0
1380,u1r1b5,1146.0
1381,u1qcxw,856.0
1382,u1qfh5,1340.0


In [358]:
frames=[data_geohash,df_neg]
geohash_cluseter=pd.concat(frames)

In [359]:
frames=[data_others,data_minus_clustered1]
data_acc1=pd.concat(frames)
data_acc1

,geohash7,nonacc_lat,nonacc_long,UJAHR,UMONAT,UWOCHENTAG,hour,geohash,cluster_id
0,u1qcuyd,52.375183,9.704767,2017,9,3,18,u1qcuy,165.0
1,u1qcu96,52.345783,9.693942,2017,7,1,10,u1qcu9,130.0
3,u1qfpx5,52.422443,9.827235,2017,1,3,23,u1qfpx,2143.0
6,u1qcu1f,52.348930,9.671276,2017,1,5,20,u1qcu1,87.0
8,u1qfp9d,52.391719,9.825786,2017,10,6,18,u1qfp9,2071.0
...,...,...,...,...,...,...,...,...,...
3674,u1qczmf,52.371667,9.814568,2017,3,6,8,u1qczm,2423.0
3675,u1qcwzc,52.338687,9.790569,2019,1,4,17,u1qcwz,2388.0
3676,u1qfq8b,52.431819,9.778349,2018,4,1,14,u1qfq8,2515.0
3677,u1qfq8c,52.432004,9.779617,2019,4,2,9,u1qfq8,2515.0


In [361]:
data_nonacc=data_acc1.copy()
data_nonacc

,geohash7,nonacc_lat,nonacc_long,UJAHR,UMONAT,UWOCHENTAG,hour,geohash,cluster_id
0,u1qcuyd,52.375183,9.704767,2017,9,3,18,u1qcuy,165.0
1,u1qcu96,52.345783,9.693942,2017,7,1,10,u1qcu9,130.0
3,u1qfpx5,52.422443,9.827235,2017,1,3,23,u1qfpx,2143.0
6,u1qcu1f,52.348930,9.671276,2017,1,5,20,u1qcu1,87.0
8,u1qfp9d,52.391719,9.825786,2017,10,6,18,u1qfp9,2071.0
...,...,...,...,...,...,...,...,...,...
3674,u1qczmf,52.371667,9.814568,2017,3,6,8,u1qczm,2423.0
3675,u1qcwzc,52.338687,9.790569,2019,1,4,17,u1qcwz,2388.0
3676,u1qfq8b,52.431819,9.778349,2018,4,1,14,u1qfq8,2515.0
3677,u1qfq8c,52.432004,9.779617,2019,4,2,9,u1qfq8,2515.0


# geoahsh labeling ends here

In [365]:
data_acc

,acc_id,acc_lat,acc_long,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,STRZUSTAND,UART,UTYP1,geohash,sessiondivided,geohash7,Gridcluster_id
0,9867_2017,52.404062,9.646150,2017,1,11,4,0,6,4,u1qf5e,10-12,u1qf5eb,1224
1,15364_2017,52.403888,9.646154,2017,5,8,3,1,5,2,u1qf5e,8-10,u1qf5eb,1224
2,15437_2017,52.401917,9.646246,2017,5,17,2,0,5,3,u1qf5e,16-18,u1qf5e2,1224
3,16196_2017,52.399327,9.646483,2017,6,15,5,0,5,3,u1qf5e,14-16,u1qf5e0,1224
4,18217_2017,52.403573,9.646172,2017,8,18,3,0,5,3,u1qf5e,18-20,u1qf5eb,1224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5659,232958_2019,52.411138,9.696425,2019,5,14,6,0,5,3,u1qfht,14-16,u1qfhth,1402
5660,232983_2019,52.368419,9.718043,2019,5,19,5,0,0,7,u1qcvj,18-20,u1qcvjk,165
5661,232984_2019,52.382647,9.710951,2019,5,18,5,0,6,1,u1qcuz,18-20,u1qcuzz,165
5662,233021_2019,52.375711,9.750454,2019,5,7,4,0,2,6,u1qcvy,6-8,u1qcvys,165


In [366]:
train_acc=data_acc.copy()
train_acc

,acc_id,acc_lat,acc_long,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,STRZUSTAND,UART,UTYP1,geohash,sessiondivided,geohash7,Gridcluster_id
0,9867_2017,52.404062,9.646150,2017,1,11,4,0,6,4,u1qf5e,10-12,u1qf5eb,1224
1,15364_2017,52.403888,9.646154,2017,5,8,3,1,5,2,u1qf5e,8-10,u1qf5eb,1224
2,15437_2017,52.401917,9.646246,2017,5,17,2,0,5,3,u1qf5e,16-18,u1qf5e2,1224
3,16196_2017,52.399327,9.646483,2017,6,15,5,0,5,3,u1qf5e,14-16,u1qf5e0,1224
4,18217_2017,52.403573,9.646172,2017,8,18,3,0,5,3,u1qf5e,18-20,u1qf5eb,1224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5659,232958_2019,52.411138,9.696425,2019,5,14,6,0,5,3,u1qfht,14-16,u1qfhth,1402
5660,232983_2019,52.368419,9.718043,2019,5,19,5,0,0,7,u1qcvj,18-20,u1qcvjk,165
5661,232984_2019,52.382647,9.710951,2019,5,18,5,0,6,1,u1qcuz,18-20,u1qcuzz,165
5662,233021_2019,52.375711,9.750454,2019,5,7,4,0,2,6,u1qcvy,6-8,u1qcvys,165


In [367]:
def geohash(lat,long):
    geo=gh.encode(float(lat), float(long), precision=7)
    return geo

In [368]:
train_acc['geohash7'] = train_acc.apply(lambda row: geohash(row['acc_lat'],row['acc_long']), axis=1)
train_acc

,acc_id,acc_lat,acc_long,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,STRZUSTAND,UART,UTYP1,geohash,sessiondivided,geohash7,Gridcluster_id
0,9867_2017,52.404062,9.646150,2017,1,11,4,0,6,4,u1qf5e,10-12,u1qf5eb,1224
1,15364_2017,52.403888,9.646154,2017,5,8,3,1,5,2,u1qf5e,8-10,u1qf5eb,1224
2,15437_2017,52.401917,9.646246,2017,5,17,2,0,5,3,u1qf5e,16-18,u1qf5e2,1224
3,16196_2017,52.399327,9.646483,2017,6,15,5,0,5,3,u1qf5e,14-16,u1qf5e0,1224
4,18217_2017,52.403573,9.646172,2017,8,18,3,0,5,3,u1qf5e,18-20,u1qf5eb,1224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5659,232958_2019,52.411138,9.696425,2019,5,14,6,0,5,3,u1qfht,14-16,u1qfhth,1402
5660,232983_2019,52.368419,9.718043,2019,5,19,5,0,0,7,u1qcvj,18-20,u1qcvjk,165
5661,232984_2019,52.382647,9.710951,2019,5,18,5,0,6,1,u1qcuz,18-20,u1qcuzz,165
5662,233021_2019,52.375711,9.750454,2019,5,7,4,0,2,6,u1qcvy,6-8,u1qcvys,165


In [369]:
train_acc=train_acc[['geohash7','acc_lat','acc_long','UJAHR','UMONAT','UWOCHENTAG','USTUNDE','Gridcluster_id']]
train_acc.columns=['geohash7','lat','long','UJAHR','UMONAT','UWOCHENTAG','hour','cluster_id']
train_acc['counts']=1
#train_acc==train_acc.reset_index()
train_acc

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,geohash7,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts
0,u1qf5eb,52.404062,9.646150,2017,1,4,11,1224,1
1,u1qf5eb,52.403888,9.646154,2017,5,3,8,1224,1
2,u1qf5e2,52.401917,9.646246,2017,5,2,17,1224,1
3,u1qf5e0,52.399327,9.646483,2017,6,5,15,1224,1
4,u1qf5eb,52.403573,9.646172,2017,8,3,18,1224,1
...,...,...,...,...,...,...,...,...,...
5659,u1qfhth,52.411138,9.696425,2019,5,6,14,1402,1
5660,u1qcvjk,52.368419,9.718043,2019,5,5,19,165,1
5661,u1qcuzz,52.382647,9.710951,2019,5,5,18,165,1
5662,u1qcvys,52.375711,9.750454,2019,5,4,7,165,1


In [370]:
geohash_cluseter=train_acc[['geohash7','cluster_id']]
geohash_cluseter.columns=['geohash','cluster_id']
geohash_cluseter

,geohash,cluster_id
0,u1qf5eb,1224
1,u1qf5eb,1224
2,u1qf5e2,1224
3,u1qf5e0,1224
4,u1qf5eb,1224
...,...,...
5659,u1qfhth,1402
5660,u1qcvjk,165
5661,u1qcuzz,165
5662,u1qcvys,165


In [372]:
dt=geohash_cluseter.groupby('cluster_id').count()
dt=dt.reset_index()
dt.to_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/geodata/clusterToGeohashCount.csv",index=False)

In [374]:
data_nonacc.columns=['geohash7','lat','long','UJAHR','UMONAT','UWOCHENTAG','hour','geohash','cluster_id']
data_nonacc=data_nonacc[['geohash7','lat','long','UJAHR','UMONAT','UWOCHENTAG','hour','cluster_id']]
data_nonacc['counts']=0
data_nonacc=data_nonacc.reset_index(drop=True)
data_nonacc

,geohash7,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts
0,u1qcuyd,52.375183,9.704767,2017,9,3,18,165.0,0
1,u1qcu96,52.345783,9.693942,2017,7,1,10,130.0,0
2,u1qfpx5,52.422443,9.827235,2017,1,3,23,2143.0,0
3,u1qcu1f,52.348930,9.671276,2017,1,5,20,87.0,0
4,u1qfp9d,52.391719,9.825786,2017,10,6,18,2071.0,0
...,...,...,...,...,...,...,...,...,...
16987,u1qczmf,52.371667,9.814568,2017,3,6,8,2423.0,0
16988,u1qcwzc,52.338687,9.790569,2019,1,4,17,2388.0,0
16989,u1qfq8b,52.431819,9.778349,2018,4,1,14,2515.0,0
16990,u1qfq8c,52.432004,9.779617,2019,4,2,9,2515.0,0


In [375]:
data_nonacc_gtminus1=data_nonacc.loc[data_nonacc['cluster_id']>-1]
data_nonacc_gtminus1

,geohash7,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts
0,u1qcuyd,52.375183,9.704767,2017,9,3,18,165.0,0
1,u1qcu96,52.345783,9.693942,2017,7,1,10,130.0,0
2,u1qfpx5,52.422443,9.827235,2017,1,3,23,2143.0,0
3,u1qcu1f,52.348930,9.671276,2017,1,5,20,87.0,0
4,u1qfp9d,52.391719,9.825786,2017,10,6,18,2071.0,0
...,...,...,...,...,...,...,...,...,...
16987,u1qczmf,52.371667,9.814568,2017,3,6,8,2423.0,0
16988,u1qcwzc,52.338687,9.790569,2019,1,4,17,2388.0,0
16989,u1qfq8b,52.431819,9.778349,2018,4,1,14,2515.0,0
16990,u1qfq8c,52.432004,9.779617,2019,4,2,9,2515.0,0


In [376]:
data_nonacc_minus1=data_nonacc.loc[data_nonacc['cluster_id']==-1]
data_nonacc_minus1

,geohash7,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts


In [378]:
frame=[train_acc,data_nonacc]
acc_non_acc=pd.concat(frame)
acc_non_acc

,geohash7,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts
0,u1qf5eb,52.404062,9.646150,2017,1,4,11,1224.0,1
1,u1qf5eb,52.403888,9.646154,2017,5,3,8,1224.0,1
2,u1qf5e2,52.401917,9.646246,2017,5,2,17,1224.0,1
3,u1qf5e0,52.399327,9.646483,2017,6,5,15,1224.0,1
4,u1qf5eb,52.403573,9.646172,2017,8,3,18,1224.0,1
...,...,...,...,...,...,...,...,...,...
16987,u1qczmf,52.371667,9.814568,2017,3,6,8,2423.0,0
16988,u1qcwzc,52.338687,9.790569,2019,1,4,17,2388.0,0
16989,u1qfq8b,52.431819,9.778349,2018,4,1,14,2515.0,0
16990,u1qfq8c,52.432004,9.779617,2019,4,2,9,2515.0,0


In [379]:
def geohash(lat,long):
    geo=gh.encode(float(lat), float(long), precision=7)
    return geo

In [382]:
geohash_cluseter=acc_non_acc[['geohash7','cluster_id']]
geohash_cluseter.columns=['geohash','cluster_id']
geohash_cluseter=geohash_cluseter.drop_duplicates()
geohash_cluseter.to_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/geohash_cluster7.csv",index=False)

In [384]:
def caculatemean(ob):
    mid=sum(ob) / len(ob) 
    return mid

In [385]:
join_hann_forlatcenter=acc_non_acc[['lat','long','cluster_id']]
latlist=join_hann_forlatcenter.groupby('cluster_id')['lat'].apply(list).reset_index(name='latlist')
latlist['middlelat']= latlist.apply(lambda row: caculatemean(row['latlist']), axis=1)
latlist

,cluster_id,latlist,middlelat
0,0.0,"[52.350046363176425, 52.35017508730086, 52.349...",52.351302
1,2.0,"[52.358087240907786, 52.356043033358056, 52.35...",52.358348
2,3.0,"[52.35316871310985, 52.353859138000075, 52.353...",52.352814
3,5.0,"[52.35669285247933, 52.35669285247933, 52.3560...",52.356783
4,9.0,"[52.3554939437576, 52.35670272019789, 52.35475...",52.355317
...,...,...,...
728,2616.0,"[52.405609558885146, 52.40530545778433, 52.405...",52.407273
729,2617.0,"[52.4132541068536, 52.41278779387968, 52.41355...",52.412861
730,2618.0,[52.41986115126147],52.419861
731,2619.0,[52.42215808116471],52.422158


In [388]:
#centroid=ob[['cluster_id','lat','long']]
longlist=join_hann_forlatcenter.groupby('cluster_id')['long'].apply(list).reset_index(name='longlist')
longlist['middlelong']= longlist.apply(lambda row: caculatemean(row['longlist']), axis=1)
longlist

,cluster_id,longlist,middlelong
0,0.0,"[9.655221825753193, 9.654434669487314, 9.65644...",9.655861
1,2.0,"[9.653997388218967, 9.65522782372234, 9.654352...",9.652560
2,3.0,"[9.661120555126367, 9.661027067000019, 9.65807...",9.660477
3,5.0,"[9.659534215163474, 9.659534215163474, 9.66076...",9.659970
4,9.0,"[9.666344717957603, 9.66543083823995, 9.666430...",9.666380
...,...,...,...
728,2616.0,"[9.909097134179332, 9.904261007174414, 9.90815...",9.905866
729,2617.0,"[9.900088650950188, 9.901905339901306, 9.90163...",9.901129
730,2618.0,[9.888476175572418],9.888476
731,2619.0,[9.888607452907976],9.888607


In [6]:
city1='grid6'

new_method/hann/clustering/gridgrowing6


In [390]:
all_merge=pd.merge(latlist,longlist,on='cluster_id')
all_merge=all_merge[['cluster_id','middlelat','middlelong']]
all_merge.to_csv('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/'+city1+"_centroid_cluster.csv",index=False)

In [391]:
all_merge.loc[all_merge['cluster_id']==-1]

,cluster_id,middlelat,middlelong


In [392]:
append_latlong=pd.merge(acc_non_acc,all_merge,on='cluster_id')
append_latlong

,geohash7,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts,middlelat,middlelong
0,u1qf5eb,52.404062,9.646150,2017,1,4,11,1224.0,1,52.400448,9.647292
1,u1qf5eb,52.403888,9.646154,2017,5,3,8,1224.0,1,52.400448,9.647292
2,u1qf5e2,52.401917,9.646246,2017,5,2,17,1224.0,1,52.400448,9.647292
3,u1qf5e0,52.399327,9.646483,2017,6,5,15,1224.0,1,52.400448,9.647292
4,u1qf5eb,52.403573,9.646172,2017,8,3,18,1224.0,1,52.400448,9.647292
...,...,...,...,...,...,...,...,...,...,...,...
22651,u1qczmf,52.371667,9.814568,2017,3,6,8,2423.0,0,52.371667,9.814568
22652,u1qcwzc,52.338687,9.790569,2019,1,4,17,2388.0,0,52.338687,9.790569
22653,u1qfq8b,52.431819,9.778349,2018,4,1,14,2515.0,0,52.431912,9.778983
22654,u1qfq8c,52.432004,9.779617,2019,4,2,9,2515.0,0,52.431912,9.778983


In [394]:
append_latlong.to_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/hann_GG_trainaccNonacc",index=False)

In [9]:
append_latlong=pd.read_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/hann_GG_train6000.csv",header=0)
append_latlong=append_latlong[['geohash7', 'lat', 'long', 'UJAHR', 'UMONAT', 'UWOCHENTAG', 'hour',
       'cluster_id', 'counts', 'middlelat', 'middlelong']]

append_latlong

,geohash7,lat,long,UJAHR,UMONAT,UWOCHENTAG,hour,cluster_id,counts,middlelat,middlelong
0,u1qfj8r,52.384881,9.743802,2017,4,3,8,165,1,52.380353,9.733844
1,u1qfj8v,52.387569,9.741408,2017,5,4,14,165,1,52.380353,9.733844
2,u1qfj8r,52.384972,9.743722,2017,5,5,17,165,1,52.380353,9.733844
3,u1qfj8z,52.387325,9.744714,2017,7,4,16,165,1,52.380353,9.733844
4,u1qfj8w,52.386426,9.742425,2017,8,2,7,165,1,52.380353,9.733844
...,...,...,...,...,...,...,...,...,...,...,...
13246,u1qfn93,52.390236,9.780429,2017,4,7,19,2493,0,52.390779,9.779383
13247,u1qfn9b,52.392561,9.778588,2018,5,4,0,2493,0,52.390779,9.779383
13248,u1qfn9b,52.392814,9.778957,2018,10,4,10,2493,0,52.390779,9.779383
13249,u1qfn93,52.390489,9.779485,2019,4,5,8,2493,0,52.390779,9.779383


In [10]:
train_generted=append_latlong[['cluster_id','UJAHR','UMONAT','UWOCHENTAG','hour','middlelat','middlelong','counts']]
train_generted

,cluster_id,UJAHR,UMONAT,UWOCHENTAG,hour,middlelat,middlelong,counts
0,165,2017,4,3,8,52.380353,9.733844,1
1,165,2017,5,4,14,52.380353,9.733844,1
2,165,2017,5,5,17,52.380353,9.733844,1
3,165,2017,7,4,16,52.380353,9.733844,1
4,165,2017,8,2,7,52.380353,9.733844,1
...,...,...,...,...,...,...,...,...
13246,2493,2017,4,7,19,52.390779,9.779383,0
13247,2493,2018,5,4,0,52.390779,9.779383,0
13248,2493,2018,10,4,10,52.390779,9.779383,0
13249,2493,2019,4,5,8,52.390779,9.779383,0


In [15]:
def getsolarpos(df,geomap):
    avgliEl=[]
    avgliZe=[]
    avgliAz=[]
    for index, row in df.iterrows():
        liZe=[]
        liAz=[]
        liEl=[]
        year=int(row['UJAHR'])
        month=int(row['UMONAT'])
        day=int(row['UWOCHENTAG'])
        hour=int(row['hour'])
        l=()
        if day==1:
            day=6
        else:
            day=day-2
        A=calendar.TextCalendar(int(day))
        lat=row['middlelat']
        long=row['middlelong']
        for k in A.itermonthdays(year,month):
            if k!=0:
                day1=date(year,month,k)
                if day1.weekday()==day:
                    dtime=str(year)+"-"+str(month).zfill(2)+"-"+str(k).zfill(2)+" "+str(hour).zfill(2)+":"+"00"+":"+"00"
                    date_time_obj = dt.datetime.strptime(dtime, '%Y-%m-%d %H:%M:%S')
                    timezone = pytz.timezone('Europe/Berlin')
                    timezone_date_time_obj = timezone.localize(date_time_obj)
                    az=get_azimuth(lat,long,timezone_date_time_obj)
                    sol_pos=pv.get_solarposition(timezone_date_time_obj,lat,long)
                    getsolarpos=sol_pos[['apparent_zenith', 'zenith', 'apparent_elevation', 'elevation','azimuth', 'equation_of_time']].values
                    ele=getsolarpos[0][3]
                    ze=getsolarpos[0][1]
                    az=getsolarpos[0][4]
                    liEl.append(ele)
                    liZe.append(ze)
                    liAz.append(az)
        npEl=np.array(liEl)
        npZe=np.array(liZe)
        npAz=np.array(liAz)
        
        avgEl=np.mean(liEl)
        avgZe=np.mean(liZe)
        avgAz=np.mean(liAz)

        avgliEl.append(avgEl)
        avgliZe.append(avgZe)
        avgliAz.append(avgAz)
        
    return avgliEl,avgliZe,avgliAz

In [16]:
from multiprocessing import Pool
import numpy as np
def create_sequences(df):
    cores = 20
    partitions = cores
    pool = Pool(cores)
    partition = int(np.ceil(float(len(df)/partitions)))    
    temp = applyParallel (df,getsolarpos,pool,partition,{'geomap':df.copy()})
    retres=temp
    print(retres)
    pool.close()
    pool.join()
    elevation = []
    zenith = []
    azimuth=[]
    for set_ in retres:
        elevation.extend(set_[0])
        zenith.extend(set_[1])
        azimuth.extend(set_[2])
    elevation, zenith,azimuth = np.array(elevation), np.array(zenith),np.array(azimuth)   
    return elevation,zenith,azimuth


In [17]:
def mergedata(geohash,year,month):
    return '20172018'

In [18]:
train_generted.columns

Index(['cluster_id', 'UJAHR', 'UMONAT', 'UWOCHENTAG', 'hour', 'middlelat',
       'middlelong', 'counts'],
      dtype='object')

In [19]:
df_midlle_latlong=train_generted[['cluster_id','middlelat','middlelong']]
df_midlle_latlong=df_midlle_latlong.drop_duplicates().reset_index().drop(['index'],axis=1)

,cluster_id,middlelat,middlelong


In [20]:
train_generted_final=train_generted.copy()
train_generted_final['is_present']=1
train_generted_final=train_generted_final.sort_values(['cluster_id','UJAHR','UMONAT','UWOCHENTAG','hour'])
train_generted_final=train_generted_final.reset_index(drop=True)
train_generted_final


,cluster_id,UJAHR,UMONAT,UWOCHENTAG,hour,middlelat,middlelong,counts,is_present
0,0,2017,2,1,3,52.351302,9.655861,0,1
1,0,2017,3,1,8,52.351302,9.655861,0,1
2,0,2017,3,5,1,52.351302,9.655861,0,1
3,0,2017,5,1,9,52.351302,9.655861,0,1
4,0,2017,5,1,11,52.351302,9.655861,0,1
...,...,...,...,...,...,...,...,...,...
13246,2498,2017,8,5,2,52.422463,9.770763,0,1
13247,2498,2017,9,1,17,52.422463,9.770763,0,1
13248,2498,2017,10,5,17,52.422463,9.770763,0,1
13249,2498,2018,2,1,7,52.422463,9.770763,0,1


In [22]:
def ispresent(middlelat,middlelong):
    is_present=-1
    if  middlelat==0 and middlelong==0:
        is_present=0
    else:
        is_present=1
    return is_present

In [23]:
import numpy as np
df_final = pd.pivot_table(train_generted_final,
                       index=['cluster_id','UJAHR','UMONAT','UWOCHENTAG','hour'],
                       values=['middlelat', 'middlelong', 'counts'],                            
                       fill_value = 0,
                       dropna=False,
                       aggfunc=np.mean).reset_index()
df_final=df_final.sort_values(['cluster_id','UJAHR','UMONAT','UWOCHENTAG','hour'])
df_final['is_present']=df_final.apply(lambda row: ispresent(row['middlelat'],row['middlelong']), axis=1)
df_final_train=df_final.loc[((df_final['UJAHR']==2017) & (df_final['UMONAT']<=12)|(df_final['UJAHR']==2018) & (df_final['UMONAT']<=12) | ((df_final['UJAHR']==2019) & (df_final['UMONAT']<=5)))]
df_final_train=df_final_train.reset_index(drop=True)
df_final_train

,cluster_id,UJAHR,UMONAT,UWOCHENTAG,hour,counts,middlelat,middlelong,is_present
0,0,2017,1,1,0,0.0,0.0,0.0,0
1,0,2017,1,1,1,0.0,0.0,0.0,0
2,0,2017,1,1,2,0.0,0.0,0.0,0
3,0,2017,1,1,3,0.0,0.0,0.0,0
4,0,2017,1,1,4,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...
1758787,2498,2019,5,7,19,0.0,0.0,0.0,0
1758788,2498,2019,5,7,20,0.0,0.0,0.0,0
1758789,2498,2019,5,7,21,0.0,0.0,0.0,0
1758790,2498,2019,5,7,22,0.0,0.0,0.0,0


In [24]:
def findpast(row):
    df_filter=pd.DataFrame(columns=['cluster_id', 'UJAHR', 'UMONAT', 'UWOCHENTAG', 'hour', 'middlelat',
       'middlelong', 'counts', 'is_present'])
    df_index = df_final_train.loc[
      (df_final_train['UJAHR'] == row['UJAHR']) & (df_final_train['UMONAT'] == row['UMONAT']) & (
                df_final_train['cluster_id'] == row['cluster_id'])
       & (df_final_train['hour'] == row['hour']) & (df_final_train['UWOCHENTAG'] == row['UWOCHENTAG'])
    &(df_final_train['is_present'] == row['is_present'])].index[0]
    i=int(df_index)-8
    for i in range(i,df_index):
         df_filter=df_filter.append((df_final_train.iloc[[i]]),ignore_index=True)
    return df_filter

In [25]:
def getdata(df_1,geomap):
    df=pd.DataFrame()
    for index, row in df_1.iterrows(): 
        temp=findpast(row)
        temp['is_p2']=0
        df=df.append(temp)
        temp1=row
        temp1['is_p2']=2
        df=df.append(temp1)
        
    return df
                                                               

In [26]:
from multiprocessing import Pool
import numpy as np
def create_sequences_1(df):
    cores = 30#Number of CPU cores on your system
    partitions = cores
    pool = Pool(cores)
    
    partition = int(np.ceil(float(len(df)/partitions)))    
    temp = applyParallel (df,getdata,pool,partition,{'geomap':df.copy()})
    retres=temp
    pool.close()
    pool.join()
    final = []
    final.extend(retres)
    return final

In [27]:
df=create_sequences_1(train_generted_final)
df

<class 'list'>
[    cluster_id UJAHR UMONAT UWOCHENTAG hour  middlelat  middlelong  counts  \
0            0  2017      1          7   19    0.00000    0.000000     0.0   
1            0  2017      1          7   20    0.00000    0.000000     0.0   
2            0  2017      1          7   21    0.00000    0.000000     0.0   
3            0  2017      1          7   22    0.00000    0.000000     0.0   
4            0  2017      1          7   23    0.00000    0.000000     0.0   
..         ...   ...    ...        ...  ...        ...         ...     ...   
4           42  2019      4          5   18    0.00000    0.000000     0.0   
5           42  2019      4          5   19    0.00000    0.000000     0.0   
6           42  2019      4          5   20    0.00000    0.000000     0.0   
7           42  2019      4          5   21    0.00000    0.000000     0.0   
441         42  2019      4          5   22   52.33702    9.693453     0.0   

    is_present  is_p2  
0            0    0.0  

[    cluster_id UJAHR UMONAT UWOCHENTAG hour  middlelat  middlelong  counts  \
 0            0  2017      1          7   19    0.00000    0.000000     0.0   
 1            0  2017      1          7   20    0.00000    0.000000     0.0   
 2            0  2017      1          7   21    0.00000    0.000000     0.0   
 3            0  2017      1          7   22    0.00000    0.000000     0.0   
 4            0  2017      1          7   23    0.00000    0.000000     0.0   
 ..         ...   ...    ...        ...  ...        ...         ...     ...   
 4           42  2019      4          5   18    0.00000    0.000000     0.0   
 5           42  2019      4          5   19    0.00000    0.000000     0.0   
 6           42  2019      4          5   20    0.00000    0.000000     0.0   
 7           42  2019      4          5   21    0.00000    0.000000     0.0   
 441         42  2019      4          5   22   52.33702    9.693453     0.0   
 
     is_present  is_p2  
 0            0    0.0  


In [28]:
df_final=pd.DataFrame()
for df1 in df:
    df_final=df_final.append(df1)
df_final=df_final.reset_index(drop=True)
df_final

,cluster_id,UJAHR,UMONAT,UWOCHENTAG,hour,middlelat,middlelong,counts,is_present,is_p2
0,0,2017,1,7,19,0.000000,0.000000,0.0,0,0.0
1,0,2017,1,7,20,0.000000,0.000000,0.0,0,0.0
2,0,2017,1,7,21,0.000000,0.000000,0.0,0,0.0
3,0,2017,1,7,22,0.000000,0.000000,0.0,0,0.0
4,0,2017,1,7,23,0.000000,0.000000,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
119254,2498,2018,6,7,7,0.000000,0.000000,0.0,0,0.0
119255,2498,2018,6,7,8,0.000000,0.000000,0.0,0,0.0
119256,2498,2018,6,7,9,0.000000,0.000000,0.0,0,0.0
119257,2498,2018,6,7,10,0.000000,0.000000,0.0,0,0.0


In [30]:
join_middle_latlong=pd.merge(df_final,df_midlle_latlong,on='cluster_id',how='left')
join_middle_latlong.drop(['middlelat_x','middlelong_x'],axis=1,inplace=True)
join_middle_latlong.columns=['cluster_id', 'UJAHR', 'UMONAT', 'UWOCHENTAG', 'hour',
       'counts','is_present','is_p2', 'middlelat', 'middlelong']
join_middle_latlong

,cluster_id,UJAHR,UMONAT,UWOCHENTAG,hour,counts,is_present,is_p2,middlelat,middlelong
0,0,2017,1,7,19,0.0,0,0.0,52.351302,9.655861
1,0,2017,1,7,20,0.0,0,0.0,52.351302,9.655861
2,0,2017,1,7,21,0.0,0,0.0,52.351302,9.655861
3,0,2017,1,7,22,0.0,0,0.0,52.351302,9.655861
4,0,2017,1,7,23,0.0,0,0.0,52.351302,9.655861
...,...,...,...,...,...,...,...,...,...,...
119254,2498,2018,6,7,7,0.0,0,0.0,52.422463,9.770763
119255,2498,2018,6,7,8,0.0,0,0.0,52.422463,9.770763
119256,2498,2018,6,7,9,0.0,0,0.0,52.422463,9.770763
119257,2498,2018,6,7,10,0.0,0,0.0,52.422463,9.770763


In [34]:
from multiprocessing import Pool
import numpy as np
def create_sequences(df):
    cores = 30#Number of CPU cores on your system
    partitions = cores
    pool = Pool(cores)
    
    partition = int(np.ceil(float(len(df)/partitions)))    
    temp = applyParallel (df,getsolarpos,pool,partition,{'geomap':df.copy()})
    retres=temp
    print(retres)
    pool.close()
    pool.join()
    elevation = []
    zenith = []
    azimuth=[]
    for set_ in retres:
        elevation.extend(set_[0])
        zenith.extend(set_[1])
        azimuth.extend(set_[2])

    elevation, zenith,azimuth = np.array(elevation), np.array(zenith),np.array(azimuth)   
    return elevation,zenith,azimuth

In [35]:
elevation,zenith,azimuth=create_sequences(join_middle_latlong)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [37]:
#city='braun_eps0.002'
join_middle_latlong['elevaion']=elevation
join_middle_latlong['zenith']=zenith
join_middle_latlong['azimuth_solar']=azimuth

In [42]:
join_middle_latlong['concate_year']='20172018'
join_middle_latlong.columns

Index(['cluster_id', 'UJAHR', 'UMONAT', 'UWOCHENTAG', 'hour', 'counts',
       'is_present', 'is_p2', 'middlelat', 'middlelong', 'elevaion', 'zenith',
       'azimuth_solar', 'concate_year'],
      dtype='object')

In [43]:
join_middle_latlong.to_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/train_final_withSolarConvareyear6000.csv",index=False)

In [45]:
l=[]
year=[]
i=0
parts1=[]
with open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/train_final_withSolarConvareyear6000.csv", 'r') as file:
    header = True
    for line in file:
        if not header:
            header = True
            continue
        parts = line.replace('\r', '').replace('\n', '').split(',')
        if [parts[0],parts[13]] not in l:
            m=[parts[0],parts[13]]
            l.append(m)
            i=0
            parts.append(i)
        else:
            i=i+1
            parts.append(i) 
        parts1.append(parts)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [46]:
import csv

def writeCsvFile(fname, data, *args, **kwargs):
    mycsv = csv.writer(open(fname, 'w'), *args, **kwargs)
    for row in data:
        mycsv.writerow(row)

writeCsvFile(r"/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/datawithazimuthUTPYE3OhannwithIndex6000.csv", parts1)

In [47]:
import pandas as pd
hann_indexed=pd.read_csv('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/datawithazimuthUTPYE3OhannwithIndex6000.csv')

hann_indexed.columns=['cluster_id', 'UJAHR', 'UMONAT', 'UWOCHENTAG', 'hour', 'counts',
       'is_present', 'is_p2', 'middlelat', 'middlelong', 'elevaion', 'zenith',
       'azimuth_solar', 'concate_year', 'timestep']
hann_indexed.to_csv('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/datawithazimuthUTPYE3OhannwithIndex6000.csv',index=False)

hann_indexed

,cluster_id,UJAHR,UMONAT,UWOCHENTAG,hour,counts,is_present,is_p2,middlelat,middlelong,elevaion,zenith,azimuth_solar,concate_year,timestep
0,0.0,2017.0,1.0,7.0,19.0,0.0,0.0,0.0,52.351302,9.655861,-20.234745,110.234745,262.672939,20172018,0
1,0.0,2017.0,1.0,7.0,20.0,0.0,0.0,0.0,52.351302,9.655861,-29.368566,119.368566,274.469019,20172018,1
2,0.0,2017.0,1.0,7.0,21.0,0.0,0.0,0.0,52.351302,9.655861,-38.337967,128.337967,287.545116,20172018,2
3,0.0,2017.0,1.0,7.0,22.0,0.0,0.0,0.0,52.351302,9.655861,-46.605055,136.605055,303.035229,20172018,3
4,0.0,2017.0,1.0,7.0,23.0,0.0,0.0,0.0,52.351302,9.655861,-53.338363,143.338363,322.355113,20172018,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119254,2498.0,2018.0,6.0,7.0,7.0,0.0,0.0,0.0,52.422463,9.770763,14.883136,75.116864,71.475077,20172018,32
119255,2498.0,2018.0,6.0,7.0,8.0,0.0,0.0,0.0,52.422463,9.770763,23.783320,66.216680,82.667311,20172018,33
119256,2498.0,2018.0,6.0,7.0,9.0,0.0,0.0,0.0,52.422463,9.770763,32.910910,57.089090,94.453730,20172018,34
119257,2498.0,2018.0,6.0,7.0,10.0,0.0,0.0,0.0,52.422463,9.770763,41.872394,48.127606,107.708087,20172018,35


In [48]:
hann_indexed.loc[hann_indexed['cluster_id']==1]

,cluster_id,UJAHR,UMONAT,UWOCHENTAG,hour,counts,is_present,is_p2,middlelat,middlelong,elevaion,zenith,azimuth_solar,concate_year,timestep


In [49]:
# creating h5 file

In [50]:
import pandas as pd
import numpy as np

import random
import os
import sys
import psutil

import matplotlib
import matplotlib.pyplot as plt
import math
from multiprocessing import cpu_count,Pool 
import multiprocessing

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MinMaxScaler
import pickle


In [52]:
def daynight(month,hour):
    if hour>=22 or (hour>=0 and hour<4):
        return 0
    elif hour>=8 and hour<=15:
        return 1
    if month==1:
        if (hour>=4 and hour<=8) or (hour>=16 and hour<=22):
            return 0
        else:
            return 1
    if month==2:
        if (hour>=4 and hour<=8) or (hour>=18 and hour<=22):
            return 0
        if hour>=16 and hour <=18:
            return 1
    if month==3:
        if (hour>=4 and hour<=6) or  (hour>=19 and hour<=22):
            return 0
        if (hour>= 7 or hour <=8) or (hour>=16 and hour <=18):
            return 1
    if month==4:
        if (hour>=4 and hour<=5) or (hour>=20 and hour<=22):
            return 0
        if (hour>= 6 and hour <=8) or (hour>= 16 and hour <=20):
            return 1
    if month==5 or month==6 or month==7 or month==8:
        if (hour>=4 and hour<=5) or (hour==22):
            return 0
        if (hour>=6 and hour<=8) or (hour>=16 and hour<=21):
            return 1 
    if month==9 or month==10:
        if (hour>=4 and hour<=7 )or (hour>=20 and hour<=22):
            return 0
        if  (hour>=16 and hour<=19) :
            return 1 
    if month==11 or month==12:
        if(hour>=4 and hour<=8) or (hour>=16 and hour<=22):
            return 0 
        
                 

In [54]:
def time_interval_two(HOD):        
    if HOD>=6 and HOD<=10 :
        return 0
    if  HOD>=11 and HOD<=15 :
        return 1
    if  HOD>=16 and HOD<=17:
        return 2;
    if HOD>=18 and HOD<=22:
        return 3
    if HOD>22 or (HOD>=0 and HOD<6):
        return 4
    else:
        return 5;

In [55]:
#Officially spring is during the months of March, April and May in Germany. Summer is from June through to August. 
#Autumn is during the months of September, October and November and winter is from December to February. 
def season(month):
    if month>=3 and month<=5:
        return 0
    if month>=6 and month<=8:
        return 1
    if month>=9 and month<=11:
        return 2
    if month==12 or month==1 or month== 2:
        return 3
    

In [57]:
geo_dict = dict(zip(sorted(hann_indexed.cluster_id.unique()), range(len(hann_indexed.cluster_id.unique()))))
f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/geodata/geo_dict.pkl","wb")
pickle.dump(geo_dict,f)
f.close()

{0.0: 0, 2.0: 1, 3.0: 2, 5.0: 3, 9.0: 4, 10.0: 5, 11.0: 6, 19.0: 7, 24.0: 8, 28.0: 9, 31.0: 10, 32.0: 11, 33.0: 12, 34.0: 13, 35.0: 14, 36.0: 15, 42.0: 16, 47.0: 17, 50.0: 18, 54.0: 19, 57.0: 20, 59.0: 21, 61.0: 22, 63.0: 23, 75.0: 24, 81.0: 25, 82.0: 26, 83.0: 27, 84.0: 28, 85.0: 29, 87.0: 30, 92.0: 31, 93.0: 32, 94.0: 33, 97.0: 34, 98.0: 35, 100.0: 36, 103.0: 37, 104.0: 38, 105.0: 39, 106.0: 40, 115.0: 41, 121.0: 42, 122.0: 43, 130.0: 44, 134.0: 45, 138.0: 46, 140.0: 47, 141.0: 48, 142.0: 49, 145.0: 50, 147.0: 51, 151.0: 52, 153.0: 53, 155.0: 54, 158.0: 55, 160.0: 56, 161.0: 57, 164.0: 58, 165.0: 59, 170.0: 60, 172.0: 61, 173.0: 62, 176.0: 63, 178.0: 64, 182.0: 65, 184.0: 66, 190.0: 67, 191.0: 68, 192.0: 69, 193.0: 70, 198.0: 71, 202.0: 72, 203.0: 73, 204.0: 74, 234.0: 75, 269.0: 76, 279.0: 77, 283.0: 78, 289.0: 79, 308.0: 80, 309.0: 81, 311.0: 82, 342.0: 83, 357.0: 84, 360.0: 85, 394.0: 86, 395.0: 87, 396.0: 88, 398.0: 89, 402.0: 90, 405.0: 91, 486.0: 92, 741.0: 93, 743.0: 94, 745.0

In [59]:
import pandas as pd
df_final=pd.read_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/datawithazimuthUTPYE3OhannwithIndex6000.csv",header=0)
df_final_train=df_final.loc[((df_final['UJAHR']==2017) & (df_final['UMONAT']<=12)|(df_final['UJAHR']==2018) & (df_final['UMONAT']<=12) | ((df_final['UJAHR']==2019) & (df_final['UMONAT']<=1)))]
df_final_valisdation=df_final.loc[((df_final['UJAHR']==2019) & (df_final['UMONAT']>1)& (df_final['UMONAT']<=5))]
df_final_train=df_final_train.reset_index(drop=True)
df_final_valisdation=df_final_valisdation.reset_index(drop=True)

In [63]:
df_final_train.columns=['geohash','year', 'month', 'day', 'twohoursession',
       'acc_count', 'is_present','is_p2', 'middlelat', 'middlelong', 'elevaion',
       'zenith', 'azimuth_solar', 'concate_year', 'timestep']
df_final_train.to_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/df_finalTrain6000.csv",index=False)

In [69]:
df_final_valisdation.columns=['geohash','year', 'month', 'day', 'twohoursession',
       'acc_count', 'is_present','is_p2', 'middlelat', 'middlelong', 'elevaion',
       'zenith', 'azimuth_solar', 'concate_year', 'timestep']
df_final_valisdation.to_csv("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/df_finalVald6000.csv",index=False)

In [74]:
def create_temporaldata(filepath,city):
    df = pd.read_csv(filepath)
    f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/geo_dict.pkl", "rb")
    geo_dict = pickle.load(f)
    f.close()
    def fun_hash(geohash):
        return geo_dict[geohash]

    df['geohash_code'] = df.apply(lambda row: fun_hash(row['geohash']), axis=1)
    def week_day(DOW):
        if DOW > 1 and DOW <= 6:
            return 1
        elif DOW == 1 or DOW == 7:
            return 0

    def make_binary(d):
        if d > 0:
            return 1
        else:
            return 0

    def year_ternary(d):
        if d == 2017:
            return 0
        if d==2018:
            return 1
        else:
            return 2

    df['DOW_cat'] = df.apply(lambda row: week_day(row['day']), axis=1)
    df['HOD_cat'] = df.apply(lambda row: time_interval_two(row['twohoursession']), axis=1)
    df['DayLight'] = df.apply(lambda row: daynight(row['month'], row['twohoursession']), axis=1)
    df['season'] = df.apply(lambda row: season(row['month']), axis=1)
    df['year'] = df.apply(lambda row: year_ternary(row['year']), axis=1)
    df['predicted_accident']=df.apply(lambda row: make_binary(row['acc_count']), axis=1)
    mergespeed_tempdf = df[[u'timestep', u'predicted_accident',u'is_p2',u'is_present', u'geohash', u'geohash_code',u'year', u'month', u'day',u'twohoursession', 
                  u'HOD_cat', u'DOW_cat', u'DayLight', u'season', u'elevaion', u'zenith', u'azimuth_solar', 'acc_count'
                           ]]
    mergespeed_tempdf.to_hdf("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/"+"ng_test.h5",
                   key='set3')
    display(mergespeed_tempdf.head())


In [75]:
print(city)

new_method/hann/clustering/gridgrowing6


In [76]:
#city='hann_clusterDBSCAN_0.003_2pts'
#city='braun'
create_temporaldata('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/df_finalVald6000.csv', city)

zero accident = 0.84670523138833


,timestep,predicted_accident,is_p2,is_present,geohash,geohash_code,year,month,day,twohoursession,HOD_cat,DOW_cat,DayLight,season,elevaion,zenith,azimuth_solar,acc_count,middlelat,middlelong
0,109,0,0.0,0.0,0.0,0,2,3.0,2.0,9.0,0,1,1,0,19.721615,70.278385,122.485822,0.0,52.351302,9.655861
1,110,0,0.0,0.0,0.0,0,2,3.0,2.0,10.0,0,1,1,0,26.781767,63.218233,136.897191,0.0,52.351302,9.655861
2,111,0,0.0,0.0,0.0,0,2,3.0,2.0,11.0,1,1,1,0,32.058882,57.941118,153.040695,0.0,52.351302,9.655861
3,112,0,0.0,0.0,0.0,0,2,3.0,2.0,12.0,1,1,1,0,34.928052,55.071948,170.733810,0.0,52.351302,9.655861
4,113,0,0.0,0.0,0.0,0,2,3.0,2.0,13.0,1,1,1,0,34.959778,55.040222,189.074319,0.0,52.351302,9.655861


# train and test

In [77]:
import pandas as pd
import numpy as np

import random
import os
import sys
import psutil

import matplotlib
import matplotlib.pyplot as plt
import math
from multiprocessing import cpu_count,Pool 
import multiprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from collections import Counter

import pickle


In [79]:
print(city)

new_method/hann/clustering/gridgrowing6


In [10]:
# hann clustering som 30x30

In [80]:
f=open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata//clustered_data_poi_acc_merged.pkl","rb")
geohash_dict = pickle.load(f)
f.close()
print(geohash_dict)

f=open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"/geodata/geo_dict.pkl","rb")
geo_dict = pickle.load(f)
f.close()
#print(geo_dict)

f=open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata//uartTrainOnly.pkl","rb")
NLP_dict_uart = pickle.load(f)
f.close()
#print(geo_dict)

f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata//zustandTrainOnly.pkl","rb")
NLP_dict_zustand = pickle.load(f)
f.close()

f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata//utypeTrainOnly.pkl","rb")
NLP_dict_utype1 = pickle.load(f)
f.close()


f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"///geodata//acc_countTrainOnly.pkl","rb")
NLP_dict_accCount = pickle.load(f)
f.close()
#print((NLP_dict_accCount))


f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata//htype.pkl","rb")
NLP_dict_htype = pickle.load(f)
f.close()

f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata//maxspeed.pkl","rb")
NLP_dict_avg_max_speed = pickle.load(f)
f.close()

f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata/hann_centerCord.pkl","rb")
NLP_dict_centerCord = pickle.load(f)
f.close()

f = open("/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/"+city+"//geodata/clusterToGeohashCountTrainOnly.pkl","rb")
NLP_dict_clustertoGeohash = pickle.load(f)
f.close()



{0.0: array([0.00087002, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00818793]), 2.0: array([0.00116003, 0.        , 0.        , 0.        , 0.00200803,
       0.        , 0.        , 0.        , 0.        , 0.00590701]), 3.0: array([0.000174  , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00608246]), 5.0: array([0.00052201, 0.        , 0.        , 0.        , 0.00133869,
       0.        , 0.        , 0.        , 0.        , 0.00807096]), 9.0: array([0.00023201, 0.        , 0.        , 0.00012771, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00120869]), 10.0: array([0.000116  , 0.        , 0.        , 0.        , 0.00334672,
       0.        , 0.        , 0.        , 0.        , 0.00241739]), 11.0: array([0.00510411, 0.        , 0.        , 0.01877395, 0.03748327,
       0.        , 0.00709579, 0.        , 0.        , 0.01329564]), 19.0: arra

In [81]:
# # Helper functions for the parallel computing 

cores = cpu_count() #Number of CPU cores on your system
partitions = 25

class WithExtraArgs(object):
     def __init__(self, func, **args):
         self.func = func
         self.args = args
     def __call__(self, df):
         return self.func(df, **self.args)

def applyParallel(data, func,pool,partition, kwargs):
    data_split = [data[i:i + partition] for i in range(0, len(data), partition)]
    data =pool.map(WithExtraArgs(func, **kwargs), data_split)
    return data

In [82]:
def onhot_enoceder_month(train):
    myEncoder = OneHotEncoder(sparse=False)
    myEncoder.fit(train['month'].values.reshape(-1, 1))

    onehot_encode = pd.concat([train.reset_index().drop('month',1),
                 pd.DataFrame(myEncoder.transform(train['month'].values.reshape(-1, 1)),
                              columns=['m_en0','m_en1','m_en2','m_en3','m_en4','m_en5','m_en6','m_en7','m_en8','m_en9','m_en10','m_en11'])], axis=1).reindex()
    return onehot_encode.drop('index',1)

In [83]:
def onhot_enoceder_day(train):
    myEncoder = OneHotEncoder(sparse=False)
    myEncoder.fit(train['day'].values.reshape(-1, 1))

    onehot_encode = pd.concat([train.reset_index().drop('day',1),
                 pd.DataFrame(myEncoder.transform(train['day'].values.reshape(-1, 1)),
                              columns=['d_en0','d_en1','d_en2','d_en3','d_en4','d_en5','d_en6'])], axis=1).reindex()
    return onehot_encode.drop('index',1)

In [84]:
def onhot_enoceder(train):
    myEncoder = OneHotEncoder(sparse=False)
    myEncoder.fit(train['HOD_cat'].values.reshape(-1, 1))

    onehot_encode = pd.concat([train.reset_index().drop('HOD_cat',1),
                 pd.DataFrame(myEncoder.transform(train['HOD_cat'].values.reshape(-1, 1)),
                              columns=['HOD_en0','HOD_en1','HOD_en2','HOD_en3','HOD_en4'])], axis=1).reindex()
    return onehot_encode.drop('index',1)

In [85]:
def scalarval(df):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(df.values.reshape(-1,1)) 
    scaled_values = scaler.transform(df.values.reshape(-1,1)) 
    df = scaled_values
    return df

In [86]:
def onhot_enoceder_season(train):
    myEncoder = OneHotEncoder(sparse=False)
    myEncoder.fit(train['season'].values.reshape(-1, 1))

    onehot_encode = pd.concat([train.reset_index().drop('season',1),
                 pd.DataFrame(myEncoder.transform(train['season'].values.reshape(-1, 1)),
                              #columns=['m_en0','m_en1','m_en2','m_en3','m_en4'])], axis=1).reindex()
                              columns=['season_0','season_1','season_2','season_3'])], axis=1).reindex()
    return onehot_encode.drop('index',1)

In [87]:
def onhot_enoceder_year(train):
    myEncoder = OneHotEncoder(sparse=False)
    myEncoder.fit(train['year'].values.reshape(-1, 1))

    onehot_encode = pd.concat([train.reset_index().drop('year',1),
                 pd.DataFrame(myEncoder.transform(train['year'].values.reshape(-1, 1)),
                              #columns=['m_en0','m_en1','m_en2','m_en3','m_en4'])], axis=1).reindex()
                              columns=['year_0','year_1','year_2'])], axis=1).reindex()
    return onehot_encode.drop('index',1)

In [94]:
def create_train_set_aug_geo(frame_list,geomap):
    process_name = str(multiprocessing.current_process())
    id = int(process_name.split(',')[0].split('-')[1])
    X_train = []
    y_train = []
    for frame in frame_list:
        training_set = frame.values
        geo_code = geo_dict[frame.geohash.iloc[0]]
        htype_count = 0
        for key, values in NLP_dict_htype.items():
            htype_count = len(values)
            break
        
        try:
            NLP_center_cord=NLP_dict_centerCord[frame.geohash.iloc[0]]
        except:
            NLP_center_cord = np.zeros(2) 
        try:
            NLP_cluster_geohash=NLP_dict_clustertoGeohash[frame.geohash.iloc[0]]
        except:
            NLP_cluster_geohash = np.zeros(1) 

        
        try:
            NLP_htype = NLP_dict_htype[frame.geohash.iloc[0]]
        except:
            NLP_htype = np.zeros(htype_count)
        try:
            geo_vec = geomap[frame.geohash.iloc[0]]
        except:
            geo_vec = np.zeros(10) 

        try:
            NLP_code_uart = NLP_dict_uart[frame.geohash.iloc[0]]
        except:
            NLP_code_uart = np.zeros(10) 

        try:
            NLP_code_uytpe1 = NLP_dict_utype1[frame.geohash.iloc[0]]
        except:
            NLP_code_uytpe1 = np.zeros(7) 

        try:
            NLP_code_zustand = NLP_dict_zustand[frame.geohash.iloc[0]]
        except:
            NLP_code_zustand = np.zeros(3) 
        
        try:
            NLP_code_max_speed = NLP_dict_avg_max_speed[frame.geohash.iloc[0]]
        except:
            NLP_code_max_speed=[0.24]
             
        try:
            NLP_code_acc_count = NLP_dict_accCount[frame.geohash.iloc[0]]
        except:
            #print('raj')
            NLP_code_acc_count = np.zeros(1) 

        
        for i in range(8, training_set.shape[0]):
            if training_set[i, 2] > 1 : 
                if training_set[i, 1] > 0:
                    a = np.concatenate((training_set[i-8:i,7:].flatten(),geo_vec),axis=0)
                    a = np.concatenate((a,NLP_code_uart),axis=0)
                    a= np.concatenate((a,NLP_code_uytpe1),axis=0)
                    a=np.concatenate((a,NLP_code_zustand),axis=0)
                    a=np.concatenate((a,NLP_code_acc_count),axis=0)
                    a=np.concatenate((a,NLP_code_max_speed),axis=0)
                    a=np.concatenate((a,NLP_htype),axis=0)
                    a=np.concatenate((a,NLP_center_cord),axis=0)
                    a=np.concatenate((a,NLP_cluster_geohash),axis=0)
                    a = np.append(a, geo_code)
                    X_train.append(a)
                    y_train.append(1)
                else:
                    a = np.concatenate((training_set[i-8:i,7:].flatten(),geo_vec),axis=0)
                    a = np.concatenate((a,NLP_code_uart),axis=0)
                    a= np.concatenate((a,NLP_code_uytpe1),axis=0)
                    a=np.concatenate((a,NLP_code_zustand),axis=0)
                    a=np.concatenate((a,NLP_code_acc_count),axis=0)
                    a=np.concatenate((a,NLP_code_max_speed),axis=0)
                    a=np.concatenate((a,NLP_htype),axis=0)
                    a=np.concatenate((a,NLP_center_cord),axis=0)
                    a=np.concatenate((a,NLP_cluster_geohash),axis=0)
                    a = np.append(a, geo_code)
                    X_train.append(a)
                    y_train.append(0)
    return X_train, y_train

In [95]:
def create_sequences(df,geohash_dict):
    frame_list=[]
    for idx, frame in df.groupby(df.geohash):        
        frame_list.append(frame)
    pool = Pool(cores)
    partition = int(np.ceil(float(len(frame_list))/partitions))    
    train_set = applyParallel (frame_list,create_train_set_aug_geo,pool,partition,{'geomap':geohash_dict.copy()})
    pool.close()
    pool.join()
    X_train = []
    y_train = []
    for set_ in train_set:
        X_train.extend(set_[0])
        y_train.extend(set_[1])

    X_train, y_train = np.array(X_train), np.array(y_train)   
    return X_train,y_train

In [96]:
print(city)

new_method/hann/clustering/gridgrowing6


In [97]:
def train_data(filename):
    df=pd.read_hdf('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'/'+'ng_train6000.h5',key='set3')    
    df=df[[u'timestep', u'predicted_accident',u'is_p2',u'is_present', u'geohash', u'geohash_code', u'month',u'day', u'twohoursession',u'year', 
                  u'HOD_cat', u'DOW_cat', u'DayLight', u'season', u'elevaion', u'zenith', u'azimuth_solar'
                           ]]
    display(df.head())
    df = onhot_enoceder_month(df)
    df= onhot_enoceder_day(df)
    df=onhot_enoceder(df)
    df=onhot_enoceder_season(df)
    df=onhot_enoceder_year(df)
    df['elevaion'] = scalarval(df['elevaion'])
    df['zenith'] = scalarval(df['zenith'])
    df['azimuth_solar'] = scalarval(df['azimuth_solar'])
    df['previous_acc']=df['predicted_accident']

    df_normalize = df.copy()
    X_train, y_train = create_sequences(df,geohash_dict)
    np.save('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'//traindata/X_train6000',X_train) 
    print (X_train.shape)
    np.save('/data/dadwal/data/DAP_data/dataPrepTrainTestCluster/Baveria/'+city+'//traindata/y_train6000',y_train)
    print( y_train.shape)
    print('done')

In [98]:
print(city)

new_method/hann/clustering/gridgrowing6


In [99]:
train_data('hann_0.003_minpts_2_poi')

whole h5 file


Index(['timestep', 'predicted_accident', 'is_p2', 'is_present', 'geohash',
       'geohash_code', 'year', 'month', 'day', 'twohoursession', 'HOD_cat',
       'DOW_cat', 'DayLight', 'season', 'elevaion', 'zenith', 'azimuth_solar',
       'acc_count', 'middlelat', 'middlelong'],
      dtype='object')

year=1
---------------------
acc > 0


,timestep,predicted_accident,is_p2,is_present,geohash,geohash_code,month,day,twohoursession,year,HOD_cat,DOW_cat,DayLight,season,elevaion,zenith,azimuth_solar
296,171,1,2.0,1.0,2.0,1,6.0,6.0,19.0,1,3,1,1,1,21.289415,68.710585,280.430557
494,153,1,2.0,1.0,3.0,2,8.0,4.0,16.0,1,2,1,1,1,40.658605,49.341395,233.184581
611,99,1,2.0,1.0,5.0,3,12.0,5.0,14.0,0,1,1,1,3,11.270231,78.729769,203.828069
629,117,1,2.0,1.0,5.0,3,12.0,7.0,2.0,0,4,0,0,3,-54.602242,144.602242,43.658525
665,153,1,2.0,1.0,5.0,3,4.0,2.0,7.0,1,0,1,1,0,4.614614,85.385386,79.654173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98547,54,1,2.0,1.0,2044.0,268,8.0,6.0,10.0,0,0,1,1,1,34.179101,55.820899,113.739345
98574,81,1,2.0,1.0,2044.0,268,3.0,2.0,15.0,1,1,1,1,0,28.633154,61.366846,219.243376
98637,144,1,2.0,1.0,2044.0,268,11.0,2.0,6.0,1,0,1,0,2,-15.365795,105.365795,100.394530
98655,162,1,2.0,1.0,2044.0,268,1.0,6.0,13.0,2,1,1,1,3,16.293490,73.706510,187.386622


,timestep,predicted_accident,is_p2,is_present,geohash,geohash_code,month,day,twohoursession,year,HOD_cat,DOW_cat,DayLight,season,elevaion,zenith,azimuth_solar
0,0,0,0.0,0.0,0.0,0,1.0,7.0,19.0,0,3,0,0,3,-20.234745,110.234745,262.672939
1,1,0,0.0,0.0,0.0,0,1.0,7.0,20.0,0,3,0,0,3,-29.368566,119.368566,274.469019
2,2,0,0.0,0.0,0.0,0,1.0,7.0,21.0,0,3,0,0,3,-38.337967,128.337967,287.545116
3,3,0,0.0,0.0,0.0,0,1.0,7.0,22.0,0,3,0,0,3,-46.605055,136.605055,303.035229
4,4,0,0.0,0.0,0.0,0,1.0,7.0,23.0,0,4,0,0,3,-53.338363,143.338363,322.355113


Index(['timestep', 'predicted_accident', 'is_p2', 'is_present', 'geohash',
       'geohash_code', 'twohoursession', 'DOW_cat', 'DayLight', 'elevaion',
       'zenith', 'azimuth_solar', 'm_en0', 'm_en1', 'm_en2', 'm_en3', 'm_en4',
       'm_en5', 'm_en6', 'm_en7', 'm_en8', 'm_en9', 'm_en10', 'm_en11',
       'd_en0', 'd_en1', 'd_en2', 'd_en3', 'd_en4', 'd_en5', 'd_en6',
       'HOD_en0', 'HOD_en1', 'HOD_en2', 'HOD_en3', 'HOD_en4', 'season_0',
       'season_1', 'season_2', 'season_3', 'year_0', 'year_1', 'year_2',
       'previous_acc'],
      dtype='object')
------------------------
#########################
process  61  started
process list with length of  15
process list with length of  15
process  62  started
process list with length of  15
process  63  started
process  65  started
process  64  started
process list with length of  15
process list with length of  15
process  66  started
process list with length of  15
process  67  started
process list with length of  15
process  68  